# Validating Responses using Regex

This notebook demonstrates how to synthesize regex-validated responses from an LLM. 

In this example, we'll ask GPT-3.5 to give us a phone number, and make sure it's just separated by dashes - so something of the format 123-456-7890.


### Define the RegEx

In [4]:
regex = '^\\d{3}-\\d{3}-\\d{4}$'

### Import Guardrails and openai

In [5]:
import openai
from guardrails import Guard
from guardrails.validators import RegexMatch
from rich import print

### Create the guard and prompt

We use a simple string guard with the RegexMatch validator. We can see that the initial response is malformatted, but the reask gets the LLM to respond correctly.

In [6]:
guard = Guard.from_string(
    validators=[RegexMatch(regex=regex, on_fail='reask')],
    description='Phone number'
)

guard(
    prompt='Generate a fake phone number',
    llm_api=openai.Completion.create
)

print(guard.state.most_recent_call.tree)

/home/zayd/workspace/shreya/guardrails/.venv/lib/python3.11/site-packages/guardrails/rail.py:115: UserWarning: Prompt must be provided during __call__.
  warnings.warn("Prompt must be provided during __call__.")
/home/zayd/workspace/shreya/guardrails/.venv/lib/python3.11/site-packages/guardrails/prompt/prompt.py:23: UserWarning: Prompt does not have any variables, if you are migrating follow the new variable convention documented here: https://docs.guardrailsai.com/0-2-migration/
  warnings.warn(
/home/zayd/workspace/shreya/guardrails/.venv/lib/python3.11/site-packages/guardrails/prompt/instructions.py:32: UserWarning: Instructions do not have any variables, if you are migrating follow the new variable convention documented here: https://docs.guardrailsai.com/0-2-migration/
  warn(


Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
│   │ │ Generate a fake phone number                                                                            │ │
│   │ │                                                                                                         │ │
│   │ │ String Output:                                                                                          │ │
│   │ │                                                                                                         │ │
│   │ │                                                                                                         │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
│   │ │ ┏━━━━━━┳━━━━━━━━━┓                                                                                      │ │
│   │ │ ┃ Role ┃ Content ┃                                                                                      │ │
│   │ │ ┡━━━━━━╇━━━━━━━━━┩                                                                                      │ │
│   │ │ └──────┴─────────┘                                                                                      │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭──────────────────────────────────────────── Raw LLM Output ─────────────────────────────────────────────╮ │
│   │ │                                                                                                         │ │
│   │ │ (919) 845-7231                                                                                          │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭─────────────────────────────────────────── Validated Output ────────────────────────────────────────────╮ │
│   │ │ FieldReAsk(                                                                                             │ │
│   │ │     incorrect_value='\n(919) 845-7231',                                                                 │ │
│   │ │     fail_results=[                                                                                      │ │
│   │ │         FailResult(                                                                                     │ │
│   │ │             outcome='fail',                                                                             │ │
│   │ │             metadata=None,                                                                              │ │
│   │ │             error_message='Result must match ^\\d{3}-\\d{3}-\\d{4}$',                                   │ │
│   │ │             fix_value='508-686-5748'                                                                    │ │
│   │ │         )                                                                                               │ │
│   │ │     ],                                                                                                  │ │
│   │ │     path=None                                                                                           │ │
│   │ │ )                                                                                                       │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯
└── ╭────────────────────────────────────────────────── Step 1 ───────────────────────────────────────────────────╮
    │ ╭────────────────────────────────────────────